In [1]:
import os
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import openai
import dotenv

In [2]:
# Configuração da API do OpenAI
dotenv.load_dotenv(dotenv.find_dotenv())
API_KEY = os.getenv("API_KEY")
openai.api_key = API_KEY
# print(openai.api_key)

In [3]:
# Configuração do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gustavo.ferreira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gustavo.ferreira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\gustavo.ferreira\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [4]:
# Função para listar todos os arquivos PDF em um diretório
def listar_pdfs_em_diretorio(diretorio):
    return [os.path.join(diretorio, arquivo) for arquivo in os.listdir(diretorio) if arquivo.lower().endswith('.pdf')]

In [5]:
# Função para converter múltiplos PDFs em texto puro
def extrair_textos_dos_pdfs(diretorio):
    lista_caminhos_pdf = listar_pdfs_em_diretorio(diretorio)
    textos = []
    for caminho_pdf in lista_caminhos_pdf:
        texto = ""
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            for pagina in leitor_pdf.pages:
                texto += pagina.extract_text()
        textos.append(texto)
    return textos

In [7]:
# Pré-processamento de texto
def preprocessar_texto(texto):
    # Tokenização
    tokens = word_tokenize(texto.lower())
    # Remoção de stopwords, pontuações e tokens não alfanuméricos
    stopwords_en = set(stopwords.words('portuguese'))
    tokens_filtrados = [token for token in tokens if token.isalnum() and token not in stopwords_en]
    # Lematização
    lemmatizer = WordNetLemmatizer()
    tokens_lemmatizados = [lemmatizer.lemmatize(token) for token in tokens_filtrados]
    if not tokens_lemmatizados:
        # Se não houver nenhuma palavra alfanumérica após o pré-processamento, manter todas as palavras
        tokens_lemmatizados = [token for token in tokens if token.isalnum()]
    return ' '.join(tokens_lemmatizados)

In [8]:
# Construção do índice invertido
def construir_indice(documentos):
    tfidf_vectorizer = TfidfVectorizer()
    matriz_tfidf = tfidf_vectorizer.fit_transform(documentos)
    return tfidf_vectorizer, matriz_tfidf

In [9]:
# Busca de documentos relevantes
def buscar_documentos(pergunta, documentos, tfidf_vectorizer, matriz_tfidf):
    pergunta = preprocessar_texto(pergunta)
    pergunta_vetor = tfidf_vectorizer.transform([pergunta]) 
    similaridades = cosine_similarity(pergunta_vetor, matriz_tfidf)
    indice_doc_relevantes = similaridades.argmax()
    return documentos[indice_doc_relevantes]

In [10]:
# Geração de respostas com o ChatGPT
def gerar_resposta(pergunta, contexto):
    prompt = f"Contexto: {contexto}\n\nPergunta: {pergunta}\n\nResposta do modelo:\n"
    resposta = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=300,
        temperature=0.5,
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    resposta_texto = resposta.choices[0]['text'].strip()
    # Limpar contexto após cada pergunta
    contexto = ""
    # Remover parênteses e aspas
    resposta_texto = resposta_texto.strip('()"')
    return resposta_texto

In [14]:
# Função principal
# Função principal
def principal():
    # 1. Pré-processamento de Dados
    diretorio_pdfs = r'C:\Users\gustavo.ferreira\Desktop\IA_chatBot\Políticas'  # Substitua pelo caminho do diretório onde os PDFs estão localizados
    textos = extrair_textos_dos_pdfs(diretorio_pdfs)
    
    # Pré-processamento dos textos dos documentos
    textos_preprocessados = [preprocessar_texto(texto) for texto in textos if texto.strip()]
    
    # 2. Indexação de Conteúdo
    tfidf_vectorizer, matriz_tfidf = construir_indice(textos_preprocessados)

    contexto = ""  # Inicializa o contexto como uma string vazia

    while True:
        # 3. Compreensão de Perguntas
        pergunta = input("Faça uma pergunta sobre os normativos corporativos (ou digite 'sair' para encerrar): ")
        if pergunta.lower() == 'sair':
            print("Até logo!")
            break

        # Adiciona o contexto à pergunta
        pergunta_com_contexto = f"{pergunta} {contexto}"

        # 4. Extração de Palavras-Chave
        palavras_chave = word_tokenize(pergunta_com_contexto.lower())

        # 5. Busca e Recuperação de Informações
        doc_relevante = buscar_documentos(pergunta_com_contexto, textos_preprocessados, tfidf_vectorizer, matriz_tfidf)

        # 6. Geração de Respostas
        resposta = gerar_resposta(pergunta_com_contexto, doc_relevante)
        print("Resposta:", resposta)

        # Atualiza o contexto com a última pergunta e resposta
        contexto += f" Pergunta: {pergunta} Resposta: {resposta}"

if __name__ == "__main__":
    principal()


Resposta: A data de criação da POLÍTICA CORPORATIVA DE REMUNERAÇÃO foi em 01/01/2019 e a última alteração foi em 01/01/2021.
Até logo!


In [13]:
diretorio_pdfs = r'C:\Users\gustavo.ferreira\Desktop\IA_chatBot\Políticas'
textos = extrair_textos_dos_pdfs(diretorio_pdfs)
print(textos)

[' POLÍTICA  CORPORATIVA DE REMUNERAÇÃO  \nAlgar S/A Empreendimentos e Participações  \nPolítica  Corporativa  Data de Criação/Alteração:  01/01 /2023 Versão: 01 \nDiretoria Corporativa  de \nGente  Criado/Alterado por:  Juliana Afonseca  Validade: Indeterminada  \n \n \n*Documento Público – Algar S/A Empreendimentos e Participações  Página 1 de 5 \n \n1. OBJETIVO  \n1.1 Estabelecer diretrizes e critérios  para a prática de Remuneração nas empresas  controladas pelo  \nGrupo Algar  (“Empresas do Grupo Algar”) , visando manter a competitividade externa, a \nmeritocracia e equidade interna , o alinhamento de interesses dos associados com a empresa e \nacionistas , e o reconhecimento exclusivamente pela performance.  \n \n2. PRINCÍPIO DO CÓDIGO DE CONDUTA ALGAR  \n2.1 Esta política  regula  os princípio s Servir e Respeito previsto s no Código de Conduta  Algar.  \n \n3. REFERÊNCIAS  \n3.1. Código de Conduta Algar ; \n3.2. Regimento Interno do Comitê de Gente;  \n3.3. Política Corporativa